[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/racousin/rag_attack/blob/main/partie1_rag_0.ipynb)

# Présentation des données VéloCorp

Ce notebook a pour but de vous présenter les données de VéloCorp et de réaliser une analyse préliminaire sur ces données. C'est la première étape de tout projet IA, qui permet par la suite de bien adapter les prompts et paramètres en fonctions du besoin métier.
Il contient les sections suivantes:
- Installation des dépendences 
- Chargement et observation des données
- Analyses préliminaires

## Installation des dépendences

In [ ]:
!pip install pandas matplotlib

In [ ]:
from typing import List, Dict
import json
import pandas as pd
import re
from google.colab import files


### Chargement et observation des données
Nous avons préparé une extraction des données de VéloCorp depuis les données textuelles (mail, documentation technique) et stocké cet extrait dans un fichier json. Cela nous permettra de nous affranchir de la complexité d'export/accès aux données pour la réalisation de notre RAG. La seconde partie du TP aura pour but de développer un agent qui pourra avoir accès directement aux bases de données afin de voir toute la chaine data.

In [ ]:
# Récupérer le fichier local
uploaded = files.upload()

# Récupérer le nom du fichier uploadé dans colab
filename = next(iter(uploaded))

In [ ]:

def manuals_json_loader(json_path: str) -> List[Dict]:
    """
    Charge uniquement les documents de type 'manuel_technique' à partir d'un fichier JSON unifié.

    Args:
        json_path (str): Chemin vers le fichier JSON fusionné contenant tous les documents.

    Returns:
        list of dict: Liste de dictionnaires contenant l'id, les métadonnées et le texte des manuels techniques.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    manuals = []  

    for key, item in data.items():
        meta = item.get("metadata", {})
        # On vérifie que le document est bien un manuel technique
        if meta and meta.get("type", "").lower() == "manuel_technique":
            manuals.append({
                "id": key,                
                "metadata": meta,         
                "text": item.get("text", "") 
            })
    return manuals

filename = "faq_manuels_mail.json"

manuals = manuals_json_loader(filename)
manuals

Voici un exemple de manuel technique pour un des produits de vélocorp.  
Essayez de retenir quelques caractérsitiques présentes (et absentes) de cette notice, cela sera utile plus tard dans le TP afin de poser des questions au RAG. 


In [ ]:
manual_id = 0 # Essayez avec différents IDs pour voir différents manuels
print(manuals[manual_id]["text"])

### Analyses préliminaires
Après nos premieres lectures des manuels, certaines caractéristiques semblent se répeter parmis tous les manuals (matériau, poids, garantie ...), essayons d'extraire ces informations de chacun des manuels afin d'améliorer notre compréhension du dataset à traiter.

In [ ]:
# Extraction des métadonnées
meta_list = []
for doc in manuals:
    meta = doc['metadata'].copy()
    meta_list.append(meta)

df = pd.DataFrame(meta_list)

# Affichage du tableau des vélos
print("Aperçu du dataset :")
df

In [ ]:
# Extraction de certaines caractéristiques numériques à titre d'exemple
def extract_poids(text):
    import re
    match = re.search(r"Poids: ([0-9]+[.,]?[0-9]*) ?kg", text)
    return float(match.group(1).replace(',', '.')) if match else None

def extract_garantie(text):
    # Cherche "Garantie: 12 mois" ou "Garantie: 2 ans"
    match = re.search(r"Garantie: *([0-9]+(?:[.,][0-9]+)?) *(mois|an|ans)", text, re.IGNORECASE)
    if match:
        valeur = float(match.group(1).replace(',', '.'))
        unite = match.group(2).lower()
        # Conversion en mois si nécessaire
        if 'an' in unite:
            valeur = valeur * 12
        return int(valeur)
    return None

def extract_material(text):
    """Optionel: utilisez ce modèle pour implémenter votre propre fonction d'extraction de caractéristiques, n'hésitez pas à utiliser chatgpt, copilot, claude pour vous aider"""
    return

# Calcul des nouvelles colonnes de caractéristiques
df['poids'] = [extract_poids(doc['text']) for doc in manuals]
df['garantie (mois)'] = [extract_garantie(doc['text']) for doc in manuals]
df['autre'] = [extract_material(doc['text']) for doc in manuals]

# Affichage des statistiques descriptives
print(df) #[['model', 'product_line', 'poids', 'garantie (mois)']])


In [ ]:
# Analysons les statistiques des caractéristiques numériques extraites
print(df.describe())

In [ ]:
# Affichage des couleurs disponibles par modèle
for i, doc in enumerate(manuals):
    import re
    match = re.search(r"Couleurs: ([^\n]+)", doc['text'])
    couleurs = match.group(1) if match else "Non spécifié"
    print(f"Modèle {doc['metadata']['model']} : couleurs disponibles -> {couleurs}")